# Libraries

In [1]:
%%capture
!pip install transformers
!pip install tdqm
#!pip install tensorflow
#!pip install torch

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

# import torch
# print(torch.__version__)

import tensorflow as tf
print(tf.__version__)

2.5.0


# Pretrained Model

In [3]:
# Tokenizer and Models
# from transformers import LongformerConfig, LongformerTokenizerFast, TFLongformerModel

# tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')
# transformer_model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096',
#                                                       output_attentions=True)
# transformer_model.config

from transformers import AutoTokenizer, TFAutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
transformer_model = TFAutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", from_pt=True,
                                               output_attentions=True)
transformer_model.config

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_attentions": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

# Data

In [25]:
# train_path = '/home/ubuntu/Longformer/train_test_data/mini_df.csv'
test_path = '/home/ubuntu/Longformer/train_test_data/mini_test_df.csv'
resample_path = '/home/ubuntu/Longformer/train_test_data/resample_df.csv'

# Get all the data
test_df = pd.read_csv(test_path)
resample_df = pd.read_csv(resample_path)

# Add separator
resample_df['Target_Text_Pair'] = list(map(lambda x,y,z: x + " " + tokenizer.sep_token + " " + y + " " + tokenizer.sep_token + " " + z, 
                                        resample_df['Target_A'], resample_df['Target_B'], resample_df['Text']))

test_df['Target_Text_Pair'] = list(map(lambda x,y,z: x + " " + tokenizer.sep_token + " " + y + " " + tokenizer.sep_token + " " + z, 
                                        test_df['TargetA'], test_df['Target_B'], test_df['Text']))

display(resample_df.head())

print('resample shape: ', resample_df.shape)
print('test shape: ', test_df.shape)

,Unnamed: 0,ID,TLINK,Target_A,Target_B,Text,End,Words,left_0,left_1,right_0,right_1,distance,Target_Text_Pair
0,0,612_SECTIME1,AFTER,further evaluation,2014-08-07,Admission Date : [R] 2014-08-07 [R] Discharge ...,end,['Admission' 'Date' ':' '[R]' '2014-08-07' '[R...,188,191,3,5,188,further evaluation [SEP] 2014-08-07 [SEP] Admi...
1,1,123_TL12,OVERLAP,NPO,IVF,Admission Date : 2013-11-21 Discharge Date : 2...,end,['Admission' 'Date' ':' '2013-11-21' 'Discharg...,65,67,69,71,2,NPO [SEP] IVF [SEP] Admission Date : 2013-11-2...
2,2,413_TL22,AFTER,anti-Candida regimen,consultation,ADMISSION DATE : 5-3-93 DISCHARGE DATE : 5-12-...,end,['ADMISSION' 'DATE' ':' '5-3-93' 'DISCHARGE' '...,173,176,81,83,95,anti-Candida regimen [SEP] consultation [SEP] ...
3,3,492_TL0,AFTER,elective coronary artery bypass grafting,Admission,[R] Admission [R] Date : 2016-02-14 Discharge ...,end,['[R]' 'Admission' '[R]' 'Date' ':' '2016-02-1...,91,97,0,2,97,elective coronary artery bypass grafting [SEP]...
4,4,337_TL19,AFTER,monitoring,referred,Admission Date : 2017-06-12 Discharge Date : 2...,end,['Admission' 'Date' ':' '2017-06-12' 'Discharg...,79,81,74,76,7,monitoring [SEP] referred [SEP] Admission Date...


resample shape:  (9000, 14)
test shape:  (11200, 14)


In [26]:
# Map labels to numbers
TLINK_map = {'AFTER':0,
             'OVERLAP': 1,
             'BEFORE': 2}

y_train = np.asarray(resample_df['TLINK'].apply(lambda x: TLINK_map[x])).reshape(-1,1)
y_test = np.asarray(test_df['TLINK'].apply(lambda x: TLINK_map[x])).reshape(-1,1)

# Distribution
print('train')
dist = np.unique(y_train, return_counts=True)[1]/sum(np.unique(y_train, return_counts=True)[1])
print(dist)

print()
print('test')
dist = np.unique(y_test, return_counts=True)[1]/sum(np.unique(y_test, return_counts=True)[1])
print(dist)

train
[0.33333333 0.33333333 0.33333333]

test
[0.10017857 0.36410714 0.53571429]


# Tokenize

In [27]:
def tokenize(max_length, sentences, tokenizer, 
             left_token=None, right_token=None, entity_mask=False, 
             global_attention=False):
    '''
    Tokenize each sentence one at a time and then batch together
    '''

    # Initialize tokens with the first example
    tokens = tokenizer.encode_plus(sentences[0], add_special_tokens=True, max_length=max_length, 
                                   padding='max_length', truncation=True, 
                                   return_tensors='tf')
    
    # Initialize entity mask with first example
    if entity_mask == True:
        left_mask = [0]*max_length
        right_mask = [0]*max_length

        # First: Index for [L] entity [L]
        idx = np.where(tokens.input_ids[0] == left_token)[0]
        for i in range(idx[0]+1, idx[1]):
            left_mask[i] = 1
        left_mask = tf.constant(left_mask, shape=(1, max_length), dtype='int32')
        tokens['left_mask'] = left_mask

        # First: Index for [R] entity [R]
        idx = np.where(tokens.input_ids[0] == right_token)[0]
        for i in range(idx[0]+1, idx[1]):
            right_mask[i] = 1
        right_mask = tf.constant(right_mask, shape=(1, max_length), dtype='int32')
        tokens['right_mask'] = right_mask
        
    # Initialize global attention mask with first example
    if global_attention == True:
        global_attention_mask = [0]*max_length
        
        # Find positions of tokens
        left_idx = np.where(tokens.input_ids[0] == left_token)[0]
        right_idx = np.where(tokens.input_ids[0] == right_token)[0]
        
        # Left and Right
        for i in range(left_idx[0]+1, left_idx[1]):
            global_attention_mask[i] = 1
         
        for i in range(right_idx[0]+1, right_idx[1]):
            global_attention_mask[i] = 1
            
        # Convert to tf.tensor  
        global_attention_mask = tf.constant(global_attention_mask, shape=(1, max_length), dtype='int32')
        tokens['global_attention_mask'] = global_attention_mask

    # Rest of the examples
    for s, sentence in tqdm(enumerate(sentences[1:])):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=max_length,
                                       padding='max_length', truncation=True, 
                                       return_tensors='tf')
        
        tokens.input_ids = tf.concat([tokens.input_ids, inputs.input_ids], 0)
        tokens.attention_mask = tf.concat([tokens.attention_mask, inputs.attention_mask], 0)

        # Rest of entity mask
        if entity_mask == True:
            left_mask = [0]*max_length
            right_mask = [0]*max_length

            # Later: Index for [L] entity [L]
            idx = np.where(tokens.input_ids[s+1] == left_token)[0]
            for i in range(idx[0]+1, idx[1]):
                left_mask[i] = 1
            left_mask = tf.constant(left_mask, shape=(1, max_length), dtype='int32')
            tokens['left_mask'] = tf.concat([tokens.left_mask, left_mask], 0)  

            # Later: Index for [R] entity [R]
            idx = np.where(tokens.input_ids[s+1] == right_token)[0]
            for i in range(idx[0]+1, idx[1]):
                right_mask[i] = 1
            right_mask = tf.constant(right_mask, shape=(1, max_length), dtype='int32')
            tokens['right_mask'] = tf.concat([tokens.right_mask, right_mask], 0) 
        
        # Rest of global attentino mask
        if global_attention == True:
            global_attention_mask = [0]*max_length
            
            # Later: left and right positions
            left_idx = np.where(tokens.input_ids[s+1] == left_token)[0]
            right_idx = np.where(tokens.input_ids[s+1] == right_token)[0]
            
            # Left and Right
            for i in range(left_idx[0]+1, left_idx[1]):
                global_attention_mask[i] = 1

            for i in range(right_idx[0]+1, right_idx[1]):
                global_attention_mask[i] = 1
                
            global_attention_mask = tf.constant(global_attention_mask, shape=(1, max_length), dtype='int32')
            tokens['global_attention_mask'] = tf.concat([tokens.global_attention_mask, global_attention_mask], 0) 

    return tokens

In [28]:
# Add special tokens
extra_tokens = ['[L]', '[R]']
tokenizer.add_tokens(extra_tokens, special_tokens=True)

left_token = tokenizer.convert_tokens_to_ids('[L]')
right_token = tokenizer.convert_tokens_to_ids('[R]')

# Resize model vocabulary
transformer_model.resize_token_embeddings(len(tokenizer)) 

print(left_token, right_token)

28996 28997


In [30]:
# Batch tokenization - 1000 examples (it) take 6 seconds 
max_length=512
global_attention=False
entity_mask=False
# X_train = tokenize(max_length, train_df['Text'], 
#                   tokenizer,
#                   left_token=left_token,
#                   right_token=right_token,
#                   entity_mask=True,
#                   global_attention=False)
# X_test = tokenize(max_length, test_df['Text'], 
#                   tokenizer,
#                   left_token=left_token,
#                   right_token=right_token,
#                   entity_mask=True,
#                   global_attention=False)
X_train = tokenize(max_length, resample_df['Target_Text_Pair'], 
                  tokenizer,
                  left_token=left_token,
                  right_token=right_token,
                  entity_mask=entity_mask,
                  global_attention=global_attention)
X_test = tokenize(max_length, test_df['Target_Text_Pair'], 
                  tokenizer,
                  left_token=left_token,
                  right_token=right_token,
                  entity_mask=entity_mask,
                  global_attention=global_attention)

print('input ids')
print(X_train.input_ids[0])
print()
if entity_mask==True:
    print('attention mask')
    print(X_train.attention_mask[0])
    print()
    print('left mask')
    print(X_train.left_mask[0])
    print()
    print('right mask')
    print(X_train.right_mask[0])
    print()
if global_attention==True:
    print('global attention mask')
    print(X_train.global_attention_mask[0])
    print()

8999it [00:35, 250.94it/s]
11199it [00:57, 193.78it/s]

input ids
tf.Tensor(
[  101  1748 10540   102  1387   118  4775   118  5004   102 10296  2236
   131 28997  1387   118  4775   118  5004 28997 12398  2236   131  1387
   118  4775   118  4925  1555   131 24928 11955  6385 10805  1607  1104
  1675  6946   131  1142  1110   170  4376   118  1214   118  1385  1299
  1443  1251  2418  1763  2657  1607  1150  2756  1106  1103  5241  1395
  1170  1103  4962 15415  1104 16320  1229  1119  1108  5569   170 11639
   119  1119  6858  1142 16320  1112  1103  4997 16320  1104  1117  1297
  1105  1125  2628 22882  1105 26979  1158  1551  1141   119  1119  1145
 10790  1104  4267 15284  1757  1120  1103  1159  1104 16320 15415   119
  1119  2756  1106  1103 14609  1324  1185  7719  1162   187  8032  1468
  6768   191  1161  2704  5241  1395   117  1187  1126  3288  1246   172
  1204 14884  1108  2373  1112  4366   119  1649   117   170   181 25509
  1197 23609 26405  5332  1108  1694   117  1107  1134  7159   122  2799
 18478  1568  1894  1892  3652

AttributeError: 

# Save numpy after tokenization

In [37]:
# np.save('X_train.input_ids', X_train.input_ids)
# np.save('X_train.attention_mask', X_train.attention_mask)
# np.save('X_train.left_mask', X_train.left_mask)
# np.save('X_train.right_mask', X_train.right_mask)

# Build Model

In [37]:
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout,\
                                    Concatenate, GlobalAveragePooling1D, \
                                    BatchNormalization

def create_model(freeze_layers=True,
                 train_size=None,
                 batch_size=1,
                 epochs=2,
                 initial_learning_rate=1e-5, 
                 final_learning_rate=1e-6, 
                 global_attention=False):
    
    # Input Layers
    input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
    attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
#     left_mask = Input(shape=(max_length,), name='left_mask', dtype='float32') 
#     right_mask = Input(shape=(max_length,), name='right_mask', dtype='float32') 
    
    if global_attention == False:
        # Transformer Layer
        bert_out = transformer_model(input_ids=input_ids,
                                    attention_mask=attention_mask) 
    
    elif global_attention == True:
        print('----Performing Global Attention----')
        global_attention_mask = Input(shape=(max_length,), name='global_attention_mask', dtype='int32') 
        
        # Transformer Layer
        bert_out = transformer_model(input_ids=input_ids,
                              attention_mask=attention_mask,
                              global_attention_mask=global_attention_mask)

    # Get embeddings and CLS token   
    X = bert_out[0]
    CLS = bert_out[1]
        
    # Get embeddings of entity pairs only
    # Approach one - dot product, only sums across token
    # left = tf.expand_dims(left_mask, -1, name='expand_left_mask')
    # right = tf.expand_dims(right_mask, -1, name='expand_right_mask')
    # left = tf.matmul(left, X, transpose_a=True, name='matmul_left_mask')
    # right = tf.matmul(right, X, transpose_a=True, name='matmul_right_mask')
    # Approach two - gather using index
#     left_idx = tf.squeeze(tf.where(left_mask[0]==1), axis=1)
#     right_idx = tf.squeeze(tf.where(right_mask[0]==1), axis=1)
#     left = tf.gather(X, left_idx, axis=1)
#     right = tf.gather(X, right_idx, axis=1)
#     left = GlobalAveragePooling1D()(left)
#     right = GlobalAveragePooling1D()(right)

#     X = Concatenate()([CLS, left, right])

    # Softmax
    X = Dropout(0.5)(CLS)
    Y = Dense(3, activation='softmax')(X)
    
    # Instantiate model
    if global_attention == False:
            model = Model(inputs=[input_ids, attention_mask], 
                          outputs = Y)
            
    elif global_attention == True:
            model = Model(inputs=[input_ids, attention_mask, left_mask, right_mask, global_attention_mask], 
                          outputs = Y)

    # Training the embeddings
    if freeze_layers == True:
        for layer in model.layers[:6]:
            layer.trainable = False
    elif freeze_layers == False:
        for layer in model.layers[:6]:
            layer.trainable = True

#     for idx, layer in enumerate(transformer_model.layers[0].encoder.layer):
#         if freeze_layers != None:
#             if idx in freeze_layers:
#                 layer.trainable = False
#             else: 
#                 layer.trainable = True
#         else:
#             layer.trainable = True

#         print(layer, layer.trainable)
            
    # Optimizer and Learning Rate Decay
    # learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/epochs)
    # steps_per_epoch = int(train_size/batch_size)
    
    # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #     initial_learning_rate=initial_learning_rate,
    #     decay_steps=steps_per_epoch,
    #     decay_rate=learning_rate_decay_factor)

    opt = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

    # Compile the model
    model.compile(optimizer=opt, 
                  loss='sparse_categorical_crossentropy')
    
    # Print model summary
    model.summary()
    
    return model

# Train 

In [47]:
del model
tf.keras.backend.clear_session()

In [38]:
from datetime import datetime
from tensorflow.keras.callbacks import History, ReduceLROnPlateau, EarlyStopping

# Decreases learning rate as loss plateaus
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, verbose=1, min_delta=1e-5)
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    verbose=2,
    mode='auto',
    baseline=None,
    restore_best_weights=True)

# Store loss on tensorboard
logdir = "/home/ubuntu/Longformer/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq=20)

# Model params:
data = X_train
labels = y_train
# class_weight = resample_class_weights

freeze_layers=False
epochs = 20
train_size = data.input_ids.shape[0]
batch_size = 4
initial_learning_rate = 1e-05
# final_learning_rate = 1e-06

# Create model and fit
model = create_model(freeze_layers=freeze_layers, 
                     epochs=epochs,
                     train_size=train_size,
                     batch_size=batch_size,
                     initial_learning_rate=initial_learning_rate)

if global_attention == False:
    output = model.fit(x=[data.input_ids, data.attention_mask], 
                      y=labels, 
                      batch_size=batch_size,
#                       class_weight=class_weight,
                      epochs=epochs,
                      validation_split=0.2,
                      callbacks=[tensorboard_callback, rlr, early_stopping_monitor])
elif global_attention == True:
    output = model.fit(x=[data.input_ids[0:-1], data.attention_mask[0:-1], 
                          data.left_mask[0:-1], data.right_mask[0:-1], 
                          data.global_attention_mask[0:-1]], 
                      y=labels[0:-1], 
#                       class_weight=class_weight,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_split=0.2,
                      callbacks=[tensorboard_callback, rlr, early_stopping_monitor])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108311808   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_40 (Dropout)            (None, 768)          0           tf_bert_model[3][13]       

# Evaluation

In [17]:
from sklearn.metrics import classification_report

pred = model.predict(x=[X_train.input_ids, X_train.attention_mask, 
                        X_train.left_mask, X_train.right_mask],
                     batch_size=4)
pred_labels = tf.argmax(pred, axis=1)

pred_df = pd.DataFrame({'Actual':y_train.ravel(), 'Predicted':pred_labels.numpy().tolist()})
print()
print(np.unique(pred_df['Predicted'], return_counts=True))
print(classification_report(pred_df['Actual'], pred_df['Predicted'], digits=3))


(array([0, 1, 2]), array([3048, 3072, 2880]))
              precision    recall  f1-score   support

           0      0.962     0.977     0.969      3000
           1      0.937     0.959     0.948      3000
           2      0.971     0.932     0.951      3000

    accuracy                          0.956      9000
   macro avg      0.956     0.956     0.956      9000
weighted avg      0.956     0.956     0.956      9000



In [40]:
from sklearn.metrics import classification_report

pred = model.predict(x=[X_test.input_ids, X_test.attention_mask],
                     batch_size=4)
pred_labels = tf.argmax(pred, axis=1)

pred_df = pd.DataFrame({'Actual':y_test.ravel(), 'Predicted':pred_labels.numpy().tolist()})
print()
print(np.unique(pred_df['Predicted'], return_counts=True))
print(classification_report(pred_df['Actual'], pred_df['Predicted'], digits=3))


(array([0, 1, 2]), array([1461, 4251, 5488]))
              precision    recall  f1-score   support

           0      0.350     0.455     0.396      1122
           1      0.718     0.749     0.733      4078
           2      0.857     0.784     0.819      6000

    accuracy                          0.738     11200
   macro avg      0.642     0.663     0.649     11200
weighted avg      0.756     0.738     0.745     11200



In [41]:
# Save predictions
results_df = test_df.copy()
results_df['labels'] = y_test.ravel()
results_df['bioclinicalbert_labels'] = pred_labels.numpy().tolist()
results_df.to_csv('baseline_pred_df.csv')

In [28]:
# Save trained model
model.save('longformer_768_20210728-044910')

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: longformer_20210726-033317/assets


INFO:tensorflow:Assets written to: longformer_20210726-033317/assets
